# DraCor API Tutorial

To use the [DraCor-API](https://dracor.org/doc/api) you need to send HTTP-Requests to the API: `https://dracor.org/api`. In Python HTTP-Request can be sent with the library [requests](https://2.python-requests.org). We have to import this library:

In [ ]:
import requests

If it fails, the package must be installed first. Run `pip install `

## `/info`: Info about the API 

In [ ]:
r = requests.get("https://dracor.org/api/info")
r.text

The API returns this information in the JSON format, which you have to parse. You can use the library `json` for this.

In [ ]:
import json
parsedResponse = json.loads(r.text)
parsedResponse

To get the current version of the API

In [ ]:
print("The current version of the Dracor-API is " + parsedResponse['version'] + ".")

## `/corpora`: list available corpora 

Load available corpora and list name and title of each corpus.

In [ ]:
r = requests.get("https://dracor.org/api/corpora?include=metrics")
corpora = json.loads(r.text)
corpora
for corpus in corpora:
    numofplays = corpus['metrics']['text']
    print(corpus['name'] + ": " + corpus['title'] + ' (' + str(numofplays) + ' plays)')

## Generic function to handle the requests and parse the result
Requesting data from the API in most cases follows a pattern:
 * construct the request-url. E.g. use `https://dracor.org/api/` as a base and attach `corpusname`, `playname`, a method, e.g. `cast` and in some cases a reponse-format, e.g. `csv`
 * use this constructed url in a request do the endpoint
 * retrieve the data and parse to a format, that can be than used in the program
 
By defining a function, this process can be speed up. Instead of repeating the code, a function can be defined, that takes `corpusname`, `playname` and `method` as arguments. In the example we assume, that the response will be JSON.

Parsing of JSON is done with the package `json`, which needs to be imported:

In [ ]:
import json

The function accepts parameters as arguments, e.g. `corpusname="ger"`. Following arguments are supported:

* `apibase` (default will be `https://dracor.org/api/`)
* `corpusname`
* `playname`
* `method`
* `parse_json`: `True`, `False` (default) – will parse the response as `json`


In [ ]:
#corpusname:str -> []
def get(**kwargs):
    #corpusname=corpusname
    #playname=playname
    #apibase="https://dracor.org/api/"
    #method=method
    #parse_json: True
    
    #could set different apibase, e.g. https://staging.dracor.org/api/ [not recommended, pls use the production server]
    if "apibase" in kwargs:
        if kwargs["apibase"].endswith("/"):
            apibase = kwargs["apibase"]
        else:
            apibase = kwargs["apibase"] + "/"
    else:
        #use default
        apibase = "https://dracor.org/api/"
    if "corpusname" in kwargs and "playname" in kwargs:
        # used for /api/corpora/{corpusname}/play/{playname}/
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"]
    elif "corpusname" in kwargs and not "playname" in kwargs:
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] 
    elif "method" in kwargs and not "corpusname" in kwargs and not "playname" in kwargs:
            request_url = apibase + kwargs["method"]
            
    else: 
        #nothing set
        request = request_url = apibase + "info"
    
    #send the response
    r = requests.get(request_url)
    if r.status_code == 200:
        #success!
        if "parse_json" in kwargs:
            if kwargs["parse_json"] == True:
                json_data = json.loads(r.text)
                return json_data
            else:
                return r.text
        else:
            return r.text
    else:
        raise Exception("Request was not successful. Server returned status code: "  + str(r.status_code))
       

The function can now be called as follows below. The function call requests the Info about the API `/api/info`:

In [ ]:
get(method="info", parse_json=True)

To request the metrics of a single play (`/api/corpora/{corpusname}/play/{playname}/metrics`) use the following function call:

In [ ]:
get(corpusname="ger",playname="lessing-emilia-galotti",method="metrics",parse_json=True)

## Example: Gender of Characters

In the following example we count characters that are tagged as "MALE" and "FEMALE in a corpus.

In [ ]:
#Get all plays in a Corpus
corpusname = input("Enter the name of a corpus, e.g. 'tat': ")
if corpusname != "":
    #get data of a single corpus and store only the list of dramas in the variable "plays"
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    #set counters for male an female characters in the corpus
    overallMale = 0
    overallFemale = 0
    #check, if a corpusname was entered
    if corpusname != "":
        #iterate over the plays
        for play in plays:
            #get the characters of a play by using the api endpoint /api/corpora/{corpusname}/play/{playname}/cast
            cast = get(corpusname=corpusname,playname=play["name"],method="cast",parse_json=True)
            #reset the counters for male and female characters
            cntMale = 0
            cntFemale = 0
            #iterate over the cast and increment the counters
            for character in cast:
                gender = character["gender"]
                if gender == "MALE":
                    cntMale = cntMale + 1
                elif gender == "FEMALE":
                    cntFemale = cntFemale + 1
            # report the result per play
            print(play["name"] + ": " + "female characters: " + str(cntFemale) + "; male characters: " + str(cntMale))
        
            # increment the overall counters
            overallMale = overallMale + cntMale
            overallFemale = overallFemale + cntFemale
    # report the results on corpus level
    print("\n\nThere are " + str(overallFemale) + " female and " + str(overallMale) + " male characters in the corpus '" + corpusname + "'")
    
else:
    raise Exception("Please enter a corpus!")
